# Bike Position - Video Analysis Tool

***
In this video analysis tool, the user can explore a recorded video of a person sitting on a bike pedalling and move from frame to frame and decide if he wants to store the image or skip ahead for further images. 

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# String manipulation and char inputs
import getch

# Display matplotlib images directly in the Jupyter notebook
%matplotlib inline

## Define input video and frame output

In [5]:
# Define input video
input_video = 'Videos/DSC_3863.mov'

# Define target name for frame output and extension type
frame_output = 'Frames/DSC_3863_'
frame_extension = '.png'

## Helper Functions

Below we define different functions used to implement the video reading and storing of frames.

In [10]:
def user_controls():
    # Initialize the user input as incorrect
    input_incorrect = True
    
    # Repeat the procedure until the user inputs a valid input
    while input_incorrect:
        # Get user input and test it for correctness
        char_input = getch.getch()
        char_input = char_input.lower()
        input_incorrect = check_user_input(char_input)
        
    # Return the correct action to be taken based on the user input
    if char_input == "a":
        video_action = -1
    elif char_input == "d":
        video_action = 1
    elif char_input == "s":
        video_action = 2
    else:
        video_action = 0
    
    # Return number corresponding to action to be taken
    # 0: close program
    # 1: move to next frame
    # -1: move to previous frame
    # 2: save frame and move to next frame
    return video_action
    
    
def check_user_input(user_input):
    # Initialize user input as incorrect
    input_incorrect = True
    
    # Define valid inputs
    valid_inputs = ["a","s","d","q"]
    
    # Check the different valid cases
    user_input = user_input.lower()
    if user_input in valid_inputs:
        input_incorrect = False
    
    # Return value
    return input_incorrect
        
    
def video_reader(input_video,frame_output,frame_extension):
    
    # Initialize frame counter
    frame_counter = 1
    counter_digits = 3
    
    # Read in the video
    video = VideoFileClip(input_video)
    video_fps = video.fps
    video_frames = video.reader.nframes
    
    # Iterate through the frames and show each frame
    continue_video = True
    current_frame = 0
    
    while continue_video and current_frame >= 0 and current_frame < video_frames:
        
        print("Entered loop")
        
        # Show current frame and wait for user input
        frame_img = video.get_frame(current_frame * video_fps)
        plt.imshow(frame_img)
        
        # Wait for user input
        
        # Testing with continue_video = False
        continue_video = False

## Read video and let user store frames

### Procedure for exploring videos and storing frames:

`d` to move to next frame

`a` to move to previous frame

`s` to save frame and move to next frame

`q` to stop reading video and exiting process


In [11]:
# Test user inputs
print(user_controls())

# Test video_reader function

# video_reader(input_video,frame_output,frame_extension)


OverflowError: character argument not in range(0x110000)